<!-- # Title here {-} -->

Per Halvorsen   \
[pmhalvor@uio.no](mailto:pmhalvor@uio.no) \
GEO4460         

# Introduction

In this paper, we build a cost-distance path between two points based on elevation and land cover data, such as snow and lakes.

The data used to build this analysis comes from the [N50 Kartdata](https://kartkatalog.geonorge.no/metadata/n50-kartdata/ea192681-d039-42ec-b1bc-f3ce04c189ac) and the [Satellittdata - Sentinel-2 - Skyfritt opptak Norge 2022](https://kartkatalog.geonorge.no/metadata/satellittdata-sentinel-2-skyfritt-opptak-norge-2022/2e996bf2-9b7b-4700-8a26-c1a8a274c136?search=satellittdata). The steps taken to generate the rasters and feature layers were executed in ArcGIS Pro.

**Cost distance** is an estimate of how difficult it is to travel from one point to another, taking into account the terrain and land cover. 
The cost-distance path is the route that minimizes the total cost of travel between two points.
The desired output of this project is the optimal path between two DNT cabins in Vestlandet, a municipality on the west coast of Norway.

Topics we explore in this analysis include:

- Joining satellite-generated orthophotos as an image mosaic
- Filtering satellite data based on values of particular bands
- Extracting barriers from land cover data
- Generating slope from elevation data
- Calculating an accumulated cost raster
- Joining rasters through basic arithmetic 
- Finding the least-cost path between two points


# Data

Inspired by the data presented in the lab and a recent ski trip from the previous winter, we decided to focus this project on the Finse region of Vestlandet.  
Instead of reusing the data from the lab, we found similar datasets on [Kartkatalog](https://kartkatalog.geonorge.no/) containing all the information needed to build this analysis.

The data sources used in this project were:

- [N50 Kartdata](https://kartkatalog.geonorge.no/metadata/n50-kartdata/ea192681-d039-42ec-b1bc-f3ce04c189ac)
- [Satellittdata - Sentinel-2 - Skyfritt opptak Norge 2022](https://kartkatalog.geonorge.no/metadata/satellittdata-sentinel-2-skyfritt-opptak-norge-2022/2e996bf2-9b7b-4700-8a26-c1a8a274c136?search=satellittdata)
- [Norges Kart](https://www.norgeskart.no/)

## [N50 Kartdata](https://kartkatalog.geonorge.no/metadata/n50-kartdata/ea192681-d039-42ec-b1bc-f3ce04c189ac)

The N50 Kartdata contains a variety of rasters, polylines, and vectors, including elevation and land cover, among other geographic features.  
Given both cabins we'll be observing lie in the same municipality, we needed to download the data for both the Vestlandet region and Buskerud region.

In the [Method](#method) section, we will go through how an elevation raster was used to create a slope raster, serving as the foundation for our cost layer.  
We'll also touch on how to extract lakes and rivers as barrier features to be used in the distance accumulation calculation.

## [Sentinel-2](https://kartkatalog.geonorge.no/metadata/satellittdata-sentinel-2-skyfritt-opptak-norge-2022/2e996bf2-9b7b-4700-8a26-c1a8a274c136?search=satellittdata)

Similar to the instructions from the lab, we'll use the cloud-free Sentinel-2 satellite imagery to create an orthophoto mosaic of the area we're interested in.  
Due to the way the cells are organized, we need to download two separate cells to properly cover the area of interest. 
Each cell covers roughly 60km x 60km. 
Even though both cabins appear in the same cell, they are very close to the edge, meaning that a "cheaper" path might exist outside just the one cell. 
Our mosaic solution ensure that our analysis is not limited to the arbitrary cell boundaries.

Image data like these are formatted as a 3-band `.tif` raster, with a red band, a green band, and a blue band.
Given that white is represented by max values for all three bands at once, we only need one of these bands to build our snow mask.
A single band can easier be loaded into the current map by double-clicking into the layer and right-clicking the desired band.
More on the process and reasoning behind using a single band in the [Method](#method) section.


## [Norges Kart](https://www.norgeskart.no/)

This source was only used to find the precise coordinates of the [Finse](https://www.norgeskart.no/#!?project=norgeskart&layers=1002&zoom=13&lat=6741397.04&lon=90229.32&markerLat=6741397.038351989&markerLon=90229.32121188036&p=searchOptionsPanel&sok=Finse) and [Krækkja](https://www.norgeskart.no/#!?project=norgeskart&layers=1002&zoom=13&lat=6723867.99&lon=99656.47&sok=kr%C3%A6kkja&markerLat=6723867.985645048&markerLon=99656.46813979524&p=searchOptionsPanel) cabins.  
The latitude and longitude coordinates for both cabins were stored as a CSV-file, which was then used to create a layer with Geo-Points. 
That process is also described in further detail in the [Method](#method) section.


# Method

The steps taken to build the cost-distance path were as follows:

- Building satellite image mosaic
- Filtering single band for snow mask
- Extracting barriers from land cover data
- Creating Geo-Points for the cabins
- Generating slope from elevation data
- Calculating an accumulated cost raster
- Joining rasters through basic arithmetic
- Finding the least cost path between two points

## Satellite data
### Building the image mosaic
Once we had downloaded the data from our source, we loaded only `Band_1` of both our current map in ArcGIS Pro. 
We used the **Mosaic To New Raster** tool to combine multiple satellite images into a single raster dataset. 
To do so, we needed to specify the input rasters, output location, desired pixel type, and number of bands (in this case, just 1). 
This step simplifies downstream processing to only handling one raster instead of two.  

### Filtering snow
As previously mentioned, we know that a pixel that is white must have a max value in all three bands, i.e. `RGB(255, 255, 255)`. 
A good approximation of where snow is then the pixels that have max values in one of the bands.
Although, considering land cover as seen from space is typically green, and water is usually blue, using max values of these colors might give a lot of false positives.
On the other hand, true red is not as common in nature.
So using red as a proxy for snow is a good approximation.
It should be noted this is a coarse simplification, to avoid having to run calculations on all three bands, which would be even more precise. 
For our analysis, we deemed this simplification acceptable.

The min and max values of digital image data are typically 0 and 255, respectively.
So we decided to use a threshold of 200, meaning that any pixel with a value greater than 200 in the red band would be classified as snow.

<!-- To filter the snow pixels, we first needed to extract the desired band from the raster. 
This was done using the **Composite Bands** tool, where we selected the red band from the joined cells of the satellite data.  -->

We used the **Raster Calculator** tool to apply the following arithmetic expression:

$$
\text{snow} = \text{Band}_1 > 200
$$

One drawback of this simpler approach is how shadows might be handled. 
Typically, there are not many shadows present in cloud-free images, but that is not always the case, as we noticed a few in our data, especially in steep areas.
Compensating for this is however outside the scope of this project, as we'll rely on high local slopes around these shadow regions to make them less likely to be included in the final least-cost path.
In a later experiment, it would be interesting to further explore the percentage of misclassified shadow pixels in the snow mask.


## N50 data 
Similar to our satellite data, we needed to download two different subsets of the N50 data, Vestlandet and Buskerud, and combine them to properly cover the area of interest here. 

### Extracting barriers from land cover data

In our N50 Kartdata, there exists a layer called `Arealdekke`, which contains information about the land cover types in the area.
We were able to select and extract the lakes and rivers from this layer using the **Select By Attributes** and **Make layer from selected attributes** tools.
The categories we selected were `Innsjø` (lake), `InnsjøRegulert` (regulated lakes) and `Elv` (river). 

### Generating slope from elevation data
The N50 data also contains a layer called **N50_Høyde_senterlinje**, which contains contour lines representing the elevation of the terrain.
From this, we were able to generate a DEM, or a digital elevation model, using the **Topo To Raster** tool.
A DEM raster is a continuous surface representing the elevation of the terrain.

Once we had the DEM, we were able to generate a raster for the slope using the **Surface Parameters** tool.
The slope raster represents the steepness of the terrain, which is important for calculating the cost of travel across the landscape.
The higher the slope, the more difficult it is to travel across the terrain, and thus the higher the cost. 

A more in-depth analysis would also consider directionality.
However, for this project, we stuck to the simplistic interpretation of straight-forward slope as a cost factor.

### Cost surface
WIth the slope raster generated, we could then move on to calculating the cost surface.
Here, we defined different weighting depending on the slope percentage, giving smaller slopes a lower cost, and an exponentially increasing cost for larger slopes.
Very large slopes get extremely high costs, which hopefully will exclude them from the least-cost path.

We used the **Raster Calculator** tool to apply the following arithmetic expression:
```
Exp("slope"/6)*("slope"<34>)+("slope">=34)*400
```

### Combined cost
With both the cost from the slope and the snow mask gerenated, we can create a combined cost raster, using the same tool as before. 
We used the **Raster Calculator** tool to apply the following arithmetic expression:
```
("snow" * 20) + ("cost" * 1)
```

## Coordinates to Geo-Points
The cabins we'll be observing are located at the following coordinates:
- Krækkja: 60.45133 N (60°27.0798′), 7.71305 (7°42.783′) E
- Finse: 60.6026197 N (60°36.1572′), 7.4916009 (7°29.4961′) E

To create the layer with geo-points for these locations, we manually wrote a CSV-file with the coordinates stored in longitude and latitude columns.
After adding this file to the current map, we used the **XY Table To Point** tool to convert the coordinates into a point layer.


## Calculating an accumulated cost raster

Use the **Distance Accumulated** tool to calculate the accumulated cost raster. 
This tool takes the cost surface raster and the barrier features, like our lakes extracted from the N50 data. 
The output is a raster where each cell represents the accumulated cost to reach that cell from the starting point, considering the barriers.

Here, we defined the **Finse hytte** as the starting point. 

## Finding the least cost path between two points
With the **Krækkja** cabin as the destination, we could now calculate the least-cost path using the **Optimal Path as Line** tool. 
We selected to generate the single best line between the two points, using the accumulated cost raster and back direction raster (a byproduct of the previous step) as inputs.
The output is a polyline representing the least-cost path between the two points, taking into account the cost surface and barriers.


# Results
The final output of this analysis is a polyline representing the least-cost path between the two cabins, Finse and Krækkja.

![Path](img/path.png)

Figure 1: The least-cost path between Finse and Krækkja cabins.

# Discussion
The resulting path seems to cut through a valley just south of Finse hytte. 
While it does cross over the top of a hill (or a mini-peak), it follows a very steady slope both up and down.
All water seems to be properly avoided as well. 

![Google path](img/google.png)

Figure 2: The path recommended by Google Maps.

When comparing the results against Google's recommended path, we can see that our path is much more direct. 
This is because Google's path follows the nearby road, RallaRvegen, which is not the most direct route between the two cabins.


![Strava path](img/strava.png)

Figure 3: The path recommended by Strava.

When compared to the route recommended by Strava, however, we can see that our expected path is very similar. 
Strava's path is a bit more direct, and our a bit more windy. 
This is likely due to our weighting for the different cost layers. 
These could likely be tuned to get results to better resemble the Strava path. 

Similar could be said about the Google path. 
Although, when it comes to building roads, water is typically not a blocker, if one can build a bridge. 
So a proper comparison for the Google path would need to consider the lakes not as barriers, but rather as a feature requiring some cost factor. 


# Summary

In this report, we presented a method for calculating the least-cost path between two points using elevation and land cover data.
We used satellite imagery to create a snow mask, and N50 data to extract barriers and generate a slope raster.
We then calculated a cost surface based on the slope and snow mask, and used this to find the least-cost path between two cabins in Vestlandet.

The resulting path was compared to paths recommended by Google Maps and Strava, showing that our method produces a reasonable approximation of the least-cost path.
The analysis could be further improved by tuning the cost factors, and by considering additional features such as roads and trails.

# References

- ArcGIS Pro: https://pro.arcgis.com/
- N50 Kartdata: https://kartkatalog.geonorge.no/metadata/n50-kartdata/ea192681-d039-42ec-b1bc-f3ce04c189ac
- Satellittdata - Sentinel-2: https://kartkatalog.geonorge.no/metadata/satellittdata-sentinel-2-skyfritt-opptak-norge-2022/2e996bf2-9b7b-4700-8a26-c1a8a274c136?search=satellittdata
- Norges Kart: https://www.norgeskart.no/

# Appendix

## Output rasters 

### Snow mask
![Snow mask](img/snow.png)

Figure 4: The snow mask generated from the satellite data.

### Slope

![Slope](img/slope.png)

Figure 5: The slope raster generated from the elevation data.


### Cost surface

![Cost surface](img/surface.png)

Figure 6: The cost surface generated from the slope data.

### Hike cost
![Hike cost](img/hikecost.png)

Figure 7: The hike cost raster generated from the cost surface and snow mask data.


### Accumulated cost
![Accumulated cost](img/dist_acc.png)

Figure 8: The accumulated cost raster generated from the cost surface and barrier data.


### Back direction
![Back direction](img/backdir.png)

Figure 9: The back direction raster generated from the accumulated cost data.